## PTSBE accuracy validation

This example checks that **PTSBE** (Pre-Trajectory Sampling with Batch Execution) produces a measurement distribution that aligns closely with **standard** (density-matrix) sampling on the same noisy circuit.

We run the **same circuit** with the **same noise model** and **same number of shots** in two ways:
1. **Standard sample** — full density-matrix simulation with noise applied at each gate.
2. **PTSBE sample** — trajectory sampling: pre-sample noise realizations and run batches of noiseless circuits.

We compare the two outcome distributions using the **Hellinger fidelity** $F$:

$$F = \sum_x \sqrt{p(x) \, q(x)}$$

where $p$ and $q$ are the empirical probabilities from each method. $F \in [0,1]$ with $F=1$ meaning identical distributions. The **Hellinger distance** is $H = \sqrt{1 - F}$. (See [Hellinger distance](https://en.wikipedia.org/wiki/Hellinger_distance) for the definitions and references.)

With a **large number of shots**, the two methods should agree to a high degree (fidelity close to 1).

In [1]:
import cudaq
import numpy as np

cudaq.set_target("density-matrix-cpu")
cudaq.set_random_seed(42)

Use a **6-qubit circuit** with three layers of Hadamards and controlled-X (CNOT) gates. For **single-qubit** gates (H) we use **DepolarizationChannel** with a single qubit in the operand list. For **CNOTs** we pass **qubit pairs** to the noise model and use **Depolarization2** (two-qubit depolarization). We use **ProbabilisticSamplingStrategy** (with a seed for reproducibility). A moderate number of **shots** (e.g. 50k–1M) gives a meaningful fidelity check across many bitstrings.

In [2]:
@cudaq.kernel
def circuit():
    n = 6
    q = cudaq.qvector(n)
    # Layer 0: Hadamards and entangling
    h(q[0])
    h(q[2])
    h(q[4])
    x.ctrl(q[0], q[1])
    x.ctrl(q[2], q[3])
    x.ctrl(q[4], q[5])
    x.ctrl(q[1], q[2])
    x.ctrl(q[3], q[4])
    # Layer 1: more H and CX
    h(q[0])
    h(q[3])
    x.ctrl(q[0], q[3])
    x.ctrl(q[2], q[5])
    x.ctrl(q[1], q[4])
    x.ctrl(q[3], q[5])
    # Layer 2: final mix
    h(q[1])
    h(q[4])
    x.ctrl(q[0], q[2])
    x.ctrl(q[1], q[5])
    x.ctrl(q[2], q[4])
    mz(q)

noise = cudaq.NoiseModel()
noise.add_channel("h", [0], cudaq.DepolarizationChannel(0.04))
noise.add_channel("h", [1], cudaq.DepolarizationChannel(0.04))
noise.add_channel("h", [2], cudaq.DepolarizationChannel(0.04))
noise.add_channel("h", [3], cudaq.DepolarizationChannel(0.04))
noise.add_channel("h", [4], cudaq.DepolarizationChannel(0.04))
noise.add_channel("h", [5], cudaq.DepolarizationChannel(0.04))
noise.add_channel("x", [0, 1], cudaq.Depolarization2(0.03))
noise.add_channel("x", [2, 3], cudaq.Depolarization2(0.03))
noise.add_channel("x", [4, 5], cudaq.Depolarization2(0.03))
noise.add_channel("x", [1, 2], cudaq.Depolarization2(0.03))
noise.add_channel("x", [3, 4], cudaq.Depolarization2(0.03))
noise.add_channel("x", [0, 3], cudaq.Depolarization2(0.03))
noise.add_channel("x", [2, 5], cudaq.Depolarization2(0.03))
noise.add_channel("x", [1, 4], cudaq.Depolarization2(0.03))
noise.add_channel("x", [3, 5], cudaq.Depolarization2(0.03))
noise.add_channel("x", [0, 2], cudaq.Depolarization2(0.03))
noise.add_channel("x", [1, 5], cudaq.Depolarization2(0.03))
noise.add_channel("x", [2, 4], cudaq.Depolarization2(0.03))

Run **standard** sampling (density-matrix with noise) and **PTSBE** sampling with the same shot count.

In [3]:
shots = 1000000

result_standard = cudaq.sample(circuit, noise_model=noise, shots_count=shots)

strategy = cudaq.ptsbe.ProbabilisticSamplingStrategy(seed=42)
result_ptsbe = cudaq.ptsbe.sample(
    circuit,
    noise_model=noise,
    shots_count=shots,
    sampling_strategy=strategy,
)

Compute the **Hellinger fidelity** between the two empirical distributions and report the result.

In [4]:
def hellinger_fidelity_from_results(result_a, result_b):
    """
    Hellinger fidelity F = sum_x sqrt(p(x)*q(x)). F in [0,1]; F=1 means identical.
    Outcomes aligned on union of bitstrings; missing outcomes treated as 0 probability.
    """
    total_a = result_a.get_total_shots()
    total_b = result_b.get_total_shots()
    if total_a == 0 or total_b == 0:
        raise ValueError("Cannot compute fidelity: one or both results have zero shots.")
    all_keys = {k for k, _ in result_a.items()} | {k for k, _ in result_b.items()}
    fidelity = 0.0
    for k in all_keys:
        p = result_a.count(k) / total_a
        q = result_b.count(k) / total_b
        fidelity += np.sqrt(p * q)
    return fidelity

fidelity = hellinger_fidelity_from_results(result_standard, result_ptsbe)
hellinger_dist = np.sqrt(max(0.0, 1.0 - fidelity))

print("PTSBE accuracy validation (standard sample vs PTSBE sample)")
print("  Strategy: ProbabilisticSamplingStrategy(seed=42)")
print("  Circuit: 6 qubits, 3 layers H/CX; DepolarizationChannel on h[0..5], Depolarization2 on CNOT pairs")
print(f"  Shots:   {shots}")
print(f"  Hellinger fidelity F:  {fidelity:.6f}")
print(f"  Hellinger distance H:  {hellinger_dist:.6f}")
min_fidelity = 0.99
if fidelity >= min_fidelity:
    print(f"  Result: F >= {min_fidelity} — PTSBE aligns well with standard sampling.")
else:
    print(f"  Result: F < {min_fidelity} — try increasing shots or re-run (stochastic).")

PTSBE accuracy validation (standard sample vs PTSBE sample)
  Strategy: ProbabilisticSamplingStrategy(seed=42)
  Circuit: 6 qubits, 3 layers H/CX; DepolarizationChannel on h[0..5], Depolarization2 on CNOT pairs
  Shots:   1000000
  Hellinger fidelity F:  0.999969
  Hellinger distance H:  0.005570
  Result: F >= 0.99 — PTSBE aligns well with standard sampling.
